In [1]:
import numpy as np
import tensorflow as tf

a_range = [0.1, 5]
b_range = [0, 2*np.pi]

n_task = 100

a = np.random.uniform(low=a_range[0], high=a_range[1], size=n_task)
b = np.random.uniform(low=b_range[0], high=b_range[1], size=n_task)

x_range = [-5, 5]
n_sample = 10

x_data = []
y_data = []
for i in range(n_task):
  x = np.random.uniform(low=x_range[0], high=x_range[1], size=n_sample)
  y = a[i] * np.sin(x + b[i])
  x = tf.reshape(x, [10, 1])
  x_data.append(x)
  y_data.append(y)

In [2]:
# Build the regression model
model = tf.keras.Sequential([tf.keras.Input(shape=(1, )),
                             tf.keras.layers.Dense(40, activation='relu'),
                             tf.keras.layers.Dense(40, activation='relu'),
                             tf.keras.layers.Dense(1)
])

#model.build(input_shape=(None ,1)) # Initiate trainable variables

model_optimizer = tf.optimizers.Adam(0.01)

In [3]:
# Get training task and testing task
n_train_task = 70
n_test_task = 30

x_train = x_data[:n_train_task]
y_train = y_data[:n_train_task]

x_test = x_data[n_train_task:]
y_test = y_data[n_train_task:]

In [8]:
# Use first order MAML to train the model
epochs = 1

for i in range(epochs):
  total_loss = 0
  for j in range(1):#n_train_task):

    # Make a deep copy for the initial variables
    meta_variables = model.trainable_variables.copy()
    print(meta_variables[0])
  
    inner_optimizer = tf.optimizers.Adam(0.1)

    # Update 2 times for the model to calculate gradients
    for m in range(2):
      with tf.GradientTape() as tape:
        y_pred = model(x_train[j])
        loss = tf.keras.losses.mean_squared_error(y_train[j], y_pred)

      gradients = tape.gradient(loss, model.trainable_variables)
      inner_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    y_pred = model(x_train[j])
    total_loss += tf.reduce_mean(tf.keras.losses.mean_squared_error(y_train[j], y_pred))
    # Normalize graidents
    n_parameters = len(meta_variables)
    for k in range(n_parameters):
      # gradients[k] = tf.math.l2_normalize(gradients[k])
      model.trainable_variables[k] = meta_variables[k]
    print(model.trainable_variables[0])
    model_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  if i % 10 == 0:
    print('Currrent Epoch is '+str(i) + '; Total loss is ' + str(total_loss.numpy()/70))

<tf.Variable 'dense/kernel:0' shape=(1, 40) dtype=float32, numpy=
array([[-0.20979877,  0.0758053 , -0.10078388,  0.07505153,  0.04500193,
        -0.11527521,  0.02489735, -0.19952464, -0.01477408,  0.17647347,
        -0.0188802 ,  0.06328756,  0.05420836, -0.18392584, -0.32840395,
         0.06104619, -0.2380152 , -0.22383572, -0.04151263,  0.0060225 ,
         0.0208026 , -0.09599284,  0.07783484, -0.0580711 , -0.17707594,
         0.06436206, -0.04209348,  0.1556262 , -0.11152793, -0.07299387,
         0.11548381,  0.02303184,  0.20319228,  0.04933336, -0.17180666,
        -0.02472497, -0.36920878,  0.05277894, -0.09873852,  0.04455147]],
      dtype=float32)>
<tf.Variable 'dense/kernel:0' shape=(1, 40) dtype=float32, numpy=
array([[-0.20979877,  0.0758053 , -0.10078388,  0.07505153,  0.04500193,
        -0.11527521,  0.02489735, -0.19952464, -0.01477408,  0.17647347,
        -0.0188802 ,  0.06328756,  0.05420836, -0.18392584, -0.32840395,
         0.06104619, -0.2380152 , -0.2238

In [24]:
# Calculate test error
test_epoch = 1000
total_test_loss = 0
for j in range(n_test_task):
  meta_variables = model.trainable_variables.copy()
  inner_optimizer = tf.optimizers.Adam(0.01)

  # Update 20 times for the model to calculate test loss
  for m in range(test_epoch):
    with tf.GradientTape() as tape:
      y_pred = model(x_test[j])
      loss = tf.keras.losses.mean_squared_error(y_test[j], y_pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    inner_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  y_pred = model(x_test[j])
  total_test_loss += tf.reduce_mean(tf.keras.losses.mean_squared_error(y_test[j], y_pred))

  n_parameters = len(meta_variables)
  for k in range(n_parameters):
    # gradients[k] = tf.math.l2_normalize(gradients[k])
    model.trainable_variables[k] = meta_variables[k]

print(total_test_loss/30)


tf.Tensor(3.7374742, shape=(), dtype=float32)


In [23]:
# Build the regression model
model_2 = tf.keras.Sequential([tf.keras.Input(shape=(1, )),
                             tf.keras.layers.Dense(40, activation='relu'),
                             tf.keras.layers.Dense(40, activation='relu'),
                             tf.keras.layers.Dense(1)
])

model_2_optimizer = tf.optimizers.Adam(0.01)
# Calculate test error
test_epoch = 1000
total_test_loss = 0
for j in range(n_test_task):
  meta_variables = model_2.trainable_variables.copy()
  inner_optimizer = tf.optimizers.Adam(0.01)

  # Update 20 times for the model to calculate test loss
  for m in range(test_epoch):
    with tf.GradientTape() as tape:
      y_pred = model_2(x_test[j])
      loss = tf.keras.losses.mean_squared_error(y_test[j], y_pred)

    gradients = tape.gradient(loss, model_2.trainable_variables)
    inner_optimizer.apply_gradients(zip(gradients, model_2.trainable_variables))
  
  y_pred = model_2(x_test[j])
  total_test_loss += tf.reduce_mean(tf.keras.losses.mean_squared_error(y_test[j], y_pred))

  n_parameters = len(meta_variables)
  for k in range(n_parameters):
    # gradients[k] = tf.math.l2_normalize(gradients[k])
    model_2.trainable_variables[k] = meta_variables[k]

print(total_test_loss/30)


tf.Tensor(3.7374747, shape=(), dtype=float32)
